In [4]:
import sys
sys.path.append('/home/ubuntu/intelliscope/modules')

#!pip install neptune-client
import os
import neptune

import tensorflow as tf
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *
from neptune.sessions import Session
import custom_metrics


################################################################################################################
project_name='Patientwise-Test-HYBRID-balanced'
prev_project_name='HYBRID'
TrainTestVal='test'
thres=0.5
################################################################################################################


NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
session = Session(api_token=NEPTUNE_API_TOKEN)

project = session.get_projects('intelliscope')['intelliscope/'+prev_project_name]
experiments=project.get_experiments()
X_test,Y_test,names = instead_data_loaders(experiments[0].get_parameters(),TrainTestVal, True) # because we have separate projects for balanced, imbalanced datasets


def get_results(model=None, X_test_pcg=None, X_test_ecg=None, Y_test=None, threshold = thres):    
    y_pred = model.predict([X_test_pcg,X_test_ecg], batch_size=32, verbose=1)
    y_test = np.argmax(Y_test,axis=1)
    
    print(custom_metrics._class_metrics(y_test, y_pred[:, 1] > threshold))
    return y_test, y_pred


def get_individual_metrics(y_test, y_pred, names):
    patient_preds={}
    for i in range(len(names)): # create dict: patient_record_name: [start_point, label, [pred0, pred1]] 
        name=names[i]
        record_name,record_start_point,_=name.strip().split('_')
        if record_name in patient_preds:
            patient_preds[record_name].append([int(record_start_point),y_test[i], list(y_pred[i])])
        else:
            patient_preds[record_name]=[[int(record_start_point),y_test[i], list(y_pred[i])]]

    y_test_bulk, y_pred_bulk=[],[]    
    for key in patient_preds: # return :: patient-wise y_test, y_preds
        patient_pred=patient_preds[key]
        label, pred_label=get_single_patient_pred(patient_pred, mean)

        y_test_bulk.append(label)
        y_pred_bulk.append(pred_label)

    y_test_bulk=np.array(y_test_bulk)
    y_pred_bulk=np.array(y_pred_bulk)
    print(custom_metrics._class_metrics(y_test_bulk, y_pred_bulk[:, 1] > thres))
    
    return y_test_bulk, y_pred_bulk

def mean(list_):
    metric=np.mean(list_)
    return [1-metric,metric]

def get_single_patient_pred(patient_pred, func):
    #print(patient_pred)
    patient_pred.sort()
    
    y=np.array(patient_pred)[:,1]
    
    assert y.sum()%(y.shape[0])==0
    label=y[0] # label of the patient
    
    rows=[]
    for row in sorted(patient_pred):
        rows.append(list(row[2]))
    rows=np.array(rows)

    y_preds=np.argmax(rows, axis=1)
    pred_label=func(y_preds)
    return label, pred_label

def neptune_log_metrics(y_test=None, y_pred=None, threshold = thres):
    log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
    log_classification_report(y_test, y_pred[:, 1] > threshold)
    log_class_metrics(y_test, y_pred[:, 1] > threshold)
    log_class_metrics_by_threshold(y_test, y_pred[:, 1])
    log_brier_loss(y_test, y_pred[:, 1])
    log_prediction_distribution(y_test, y_pred[:, 1])

    log_log_loss(y_test, y_pred)
    log_roc_auc(y_test, y_pred)
    log_precision_recall_auc(y_test, y_pred)
    log_ks_statistic(y_test, y_pred)
    log_cumulative_gain(y_test, y_pred)
    log_lift_curve(y_test, y_pred)


Total  388
197 Images loaded across 2 Categories. Ignored blanks  0
191 Images loaded across 2 Categories. Ignored blanks  16
372 Total After removing blanks


In [ ]:

X_test_pcg = X_test[:,:,:,0:3]/255.0
X_test_ecg = X_test[:,:,:,3:6]/255.0

for i in range(len(experiments)):
    if i==0:continue
    experiment    = experiments[i]
    model_name = str(experiment)+'.h5'
    PARAMS     = experiment.get_parameters()
    PARAMS['tags']=PARAMS['tags'][2:-2].strip().split("', '")
    
    model = load_model(PARAMS['modelsave_dir']+'/'+model_name)
    y_test, y_pred=get_results(model, X_test_pcg, X_test_ecg, Y_test)
    
    y_test_patientwise, y_pred_patientwise=get_individual_metrics(y_test, y_pred, names)

    
    neptune.init('intelliscope/'+project_name,NEPTUNE_API_TOKEN)
    exp = neptune.create_experiment(name=str(experiment),description=PARAMS['name'],params=PARAMS,tags=PARAMS['tags'],upload_stdout=True)
    neptune_log_metrics(y_test_patientwise, y_pred_patientwise)
    neptune.stop() 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.7419354838709677, 'precision': 0.7135135135135136, 'recall': 0.7542857142857143, 'f1_score': 0.7333333333333335, 'f2_score': 0.7457627118644068, 'matthews_corrcoef': 0.48440785428865124, 'cohen_kappa': 0.48370680931039467, 'true_positive_rate': 0.7542857142857143, 'true_negative_rate': 0.7309644670050761, 'positive_predictive_value': 0.7135135135135136, 'negative_predictive_value': 0.7700534759358288, 'false_positive_rate': 0.26903553299492383, 'false_negative_rate': 0.24571428571428572, 'false_discovery_rate': 0.2864864864864865}
{'accuracy': 0.75, 'precision': 0.75, 'recall': 0.7142857142857143, 'f1_score': 0.7317073170731706, 'f2_score': 0.7211538461538461, 'matthews_corrcoef': 0.4984447862792268, 'cohen_kappa': 0.4979253112033195, 'true_positive_rate': 0.7142857142857143, 'true_negative_rate': 0.782608695652174, 'positive_predictive_value': 0.75, 'negative_predictive_value': 0.75, 'false_positive_rate': 0.21739

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/Patientwise-Test-HYBRID-Balanced/e/PWTESTBAL-1
